<a href="https://colab.research.google.com/github/mm002a/JupyterNotebooks-V0.0/blob/main/Leaks1_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational Leak Detection: The Plan
#### V1.20 (20/8/24)

1. Simple case studies, just do a few to see what is going on.
   *  Try working through a few simple cases by hand.
1. Definition of leaking pipe object
   1. What is the minimal set of assumptions that are required to make this work?
      1. Four readings are available on a pipe:
  $\dot{M}_{in}$, $P_{in}$ and $\dot{M}_{out}$, $P_{out}$
      1. Assume that the leak rate
  $\dot{Q}_{leak} \propto ( P_{leak}-P_{ext})^2$
      1. Assume the external pressure $P_{ext}$ is constant.
      1. Assume we have an  **accurate** *pressure-drop/flow-rate/distance* correlation:
      ${\cal F}(\dot{M},P_{in},P_{out},L) = 0$
1. Exploration of known uncertainty
   1. How does meter uncertainty translate to leak uncertainty?
   1. How does a single leak compare to multiple leaks? (Hypothesis, there is an equivalence between one leak and two smaller ones; can this be verified?)
1. Include (e.g. diurnal) variation - this will lead to separating variations due to leaks and other variations e.g.:
   1. Changes in flow properties
   1. Operating conditions
   1. (Say) thermal effects
1. Use ML (=TensorFlow?) to improve equation solution.
1. Look at networks - what is the effect of condition number?

### Status
1. FUNCTIONS:
   1. **WeymouthPP** (?redundant)(removed)
   1. **Weymouth(Qdot, Pin, Pout, L, D, Sg;PARAMS)** - returns one of first 4 args, given other 3.
   1. FindLeakInit (redundant)(removed)
   1. Define the `DATASTRUCTURES` for a set of __LEAKS__:
      - A Leak = $[LLeak,C]$ is an ordered pair {distance from the inlet of the pipe, the constant C}
      (recall: $\dot{M}_{leak} = C(P_{leak} - P_{external})^2$ )
      - A LeakSet is a list of Leaks: $[\cdots[LLeak_n,C_n]\cdots]$
      - Note that `len([]) = 0` and `len([[LL1,C1]]) = 1` and `len([[LL1,C1],[LL2,C2]]) = 2`
      - **SO** `len()` is all we need to determine the number of LEAKS!  
      (But must be used carefully because: `len([LL1,C1]) = 2` (**ALAS**!)
   1. GenLeak - needs writing!
   1. **FindLoc(LLeak, PARAMS)** - returns (`error = Pout - PoutEst`)
      * Validated?
   1. **FindLeak** Validated? - needs cleaning-up
   1. **CheckLeak(Pin,PLeak,Pout,LLeak;PARAMS)** - returns `MLeakTest` & prints some info.
      * Validated? - needs cleaning up
1. **Clean-up and validate further to get to V1.1**
   * Extend by varying `Pin` ($=P_{in}$). Set `Pext` ($=P_{external} = 1bara$).
   * Extend by trying to resolve multiple leaks using data from multiple flow rates (different values of `Pin`).
1. V1.3 include metering uncertainty.
1. V2.0 decrease uncertainty by using external pressure and multiple flow rates.
1. **Fork** What about using induction?   
__Assume:__
   - We can solve a "system" with two nodes and one pipe. (This is the easy starting condition.)
   - We can grow a solved system  by:
      1. adding a node in the middle of a pipe (a new leak) -- this **replaces** one pipe with two (sub-pipes) and an **unknown source** which satisfies the leak flow rate.
      1. add a new pipe and node (a 'conventionally' bigger system)
   - We interogate the system by:
      1. running the system at different flow rates
      1. adding pressure sensors
      1. adding flow meters
This needs some work.
   1. Define $D$ to be a DAG (directed acyclic graph) or maybe a tree. Define the DAG by: $D(n,e)$ where the number of nodes is: $n$, the number of edges is: $e$.
      1. Clearly increasing the size of the network (either by conventionally increasing the network or by hypothesising a new leak) we have: $D(n,e) \rightarrow D(n+1, e+1)$
      - If we have a leak then we have constraints on the length of the edges
   $L^{broken} = L^{new}_1 +L^{new}_2$
      and the flow rate at the new node:
  $Q(Node^{new}) =C_i(P({Node}^{new}) - P_{external})^{\alpha}$.

#### Stage 1.1: Single pipe - *N*-leaks.
Some notation:
1. Define the position of leak $i$ by $L_i$.
   - $L_0$ is the inlet and $L_{N+1}$ is the outlet. Of course, $L_0 = 0$ and the pipe length, $L_{N+1}$ is known. _Unknowns: $N$_
1. We assume that we have data for the flow performance for $R$ different inlet pressures. The data is the inlet and outlet pressures and flow rates:  
$P_0(r) , P_{N+1}(r), \dot{Q}_0(r) , \dot{Q}_{N+1}(r)$ respectively.
_"Equations": 4R (discount)_
1. With each pipe segment, $L_{i-1}$ to $L_i$, associate a flowrate $\dot{Q}_{i-1}(r)$
 $i = 1\dots N, r=1\dots R$
1. With each node ($L_i$) we have a leak coefficient $C_i$. _Unknowns: $N$_
1. Further, for each node and for each flow case ($r$),  we associate a pressure, $P_i(r)$ and a leakage rate, $\dot{Q}_i^{leak}(r)$.
  - Obviously mass conservation at node $i$ gives:
  $$
  \dot{Q}_i^{leak}(r) = \dot{Q}_i(r) - \dot{Q}_{i-1}(r)
  $$

We have the folling relationships:
1. We assume that the leakage rate is determined by Bernoulli's equation; so
$$
\dot{Q}_i^{leak}(r) = C_i \left[P_i(r) - P_{ext}\right]^2
$$
where $P_{ext}$ is the external pressure, assumed consant for values of $i$ and $r$.
1. The pressure drop along segment $L_{i-1}$ to $L_i$ satisfies:
$$
{\cal F}\left(\overline{\rho}_{i-1}(r)\dot{Q}_{i-1}(r),P_{i-1}(r),P_{i}(r),(L_{i-1}-L_i)\right) = 0
$$


The next question - how do we find *N*?
Approach:
- Loop: NLeak = 1,NLkMax monitor size of the normalized residual.
- Compare:
    - Infinity norm (look at some components - e.g. all $P_i$, $Q_i$ and _ALL_ equations).
    - Normalised two norm, as we did with PAMG -?- $\left (\frac{1}{NLeak}\sum_{NLeak} \left. res_{P_i}^2 \right. \right)$
    - **How does this quantity vary?**

#### Stage 2: Single pipes and uncertainty.
-   In the presence of uncertainty we have uncertainty in the sensor readings: $\dot{M}_{in}$, $P_{in}$ and $\dot{M}_{out}$, $P_{out}$
   -  Assume they are all uniformly distributed with a parameter ${\mathbf{\chi}}$.  
   Values of $\chi$ can be obtained from the equipment manufacturer (see above).
- Denote the uncertain readings by: $\overline{\dot{M}}_{x}, \overline{P}_{x}$ at location $x \in \{in, leak, out\}$.
Then we have:
   -  $\overline{\dot{M}}_{leak} = \overline{\dot{M}}_{in} - \overline{\dot{M}}_{out}$
   -  ${\cal W}(\overline{\dot{M}}_{in},\overline{P}_{in},\overline{P}_{leak},\overline{L}_{leak}) = 0$
   -  ${\cal W}(\overline{\dot{M}}_{out},\overline{P}_{leak},\overline{P}_{out},(L - \overline{L}_{leak})) = 0$
   -  $\overline{\dot{M}}_{leak} = C(\overline{P_{leak}} - P_{external})^2$
- Let us assume that we have $n$ uncertainty readings denoted by:  $\overline{\dot{M}}_{x}(i), \overline{P}_{x}(i)$ at location $x \in \{in, leak, out\}$; $ 1 \leq i \leq n$.  
Then, for each $i$ we can write:
   -  $\overline{\dot{M}}_{leak}(i) = \overline{\dot{M}}_{in}(i) - \overline{\dot{M}}_{out}(i)$
   -  ${\cal W}(\overline{\dot{M}}_{in}(i),\overline{P}_{in}(i),\overline{P}_{leak}(i),\overline{L}_{leak}(i)) = 0$
   -  ${\cal W}(\overline{\dot{M}}_{out}(i),\overline{P}_{leak}(i),\overline{P}_{out},(L - \overline{L}_{leak}(i))) = 0$
   -  $\overline{\dot{M}}_{leak}(i) = C(\overline{P_{leak}}(i) - P_{external})^2$
we are assuming that: $C$ and $P_{external}$ are unaffected by the flow paramters.

## Leaks - some explorations

### Other materials
-  [CALDON Meters][1] (single phase liquid). This link specifies an error of __0.01%__ throughout the range.
[1]: https://www.sensiaglobal.com/Measurement/Types/Ultrasonic-Flowmeters/Liquid-Hydrocarbon-Ultrasonic-Flowmeters/CALDON-LEFM-280Ci-Ultrasonic-Flow-Meter

### Case - simple pipe, single leak.
- Use the Weymouth Equation to calculate flow rates (it can be any correlation!). For completenesss the Weymouth equation is given by:    
$$
\dot{Q}_{st} = 137.2364 \frac{T_{st}}{P_{st}}
\left [ \frac{P_1^2-P_2^2-E_{PE}}{L ~S_g~ T_{avg}z_{avg}} \right ] ^{1/2}
D^{8/3}
$$
- Inputs from sensors: $\dot{M}_{in}$, $P_{in}$ and $\dot{M}_{out}$, $P_{out}$
- Physical parameters $D (= 0.34)$, $L (= 1.65E5)$, $S_g (= 0.693)$, $Z_{avg} = 1$, $T_{st} = 288.15$, $P_{st} = 1.01325E5$, $E_{PE} = 0$

- Flow parameters: $\dot{M}_{in} = 34.25483928234368$, $P_{in} = 9.0 {\rm MPa}$ and $\dot{M}_{out} = 34.25483928234368$, $P_{out} = 2 {\rm MPa}$

Topography:

   [$\dot{M}/P_{in}$ ----------------------------------- $\dot{M}/P_{leak}$ --------------------------------$\dot{M}/P_{out}$]
   
   |< -------------------------$L_{leak}$---------------->|

                             External Pressure $P_{ext}$.
                             
We assume that:
-  we know an accurate correlation between flow rate and pressure drop (in this case Weymouth: ${\cal W}\left(\dot{M},P_{in},P_{out},L\right) = 0$).
-  there is a known correlation $\dot{M}_{leak} = {\cal F}_{leak}(P_{leak} - P_{external})$ -  
(we'll start with $\dot{M}_{leak} = C(P_{leak} - P_{external})^2$).

#### Stage 1: Single pipe - single leak.
Clearly:
-   $M_{leak} = M_{in} - M_{out}$
-   We need to solve:
   -       ${\cal W}\left(\dot{M}_{in},P_{in},P_{leak},L_{leak}\right) = 0$
   -       ${\cal W}\left(\dot{M}_{out},P_{leak},P_{out},(L_{tot} - L_{leak})\right) = 0$
   -       for $P_{leak},L_{leak}$.
   - __SOLVED 20/7/20 in:__ `Leaks1.py`
      - NEEDS:--
      1.   Cleaning up (some done `Leaks1.1.ipynb`).
      2.   Validating :-). The accuracy seems very good (for single pipes)
      3.   Write $\rho$ as $\rho(P,T)$ from  `GasCompressibility.ipynb`

In [ ]:
#Introduce L as an argument (not a PARAMETER) for the leaks work.
# 1) Swap the arguments 2) Validate 3) Change the parameters 4) Validate. == DONE. IS MainArgs and PARAMS used??
#DEFINE Generalised Transport Equations as RetCode = f(Qdot, Pin, Pout; parameters)
#Syntax: OUTPUT varb is initialised to -9999.
#This works because, in real life everything must be positive.
# Triple:{Qdot, Pin, Pout} legal example: {-9999, Pin>Pout, Pout>0}- find Qdot, {Qin>0,-Pin,-9999} find Pout by varying Pin
def Weymouth(Qdot, Pin, Pout, L, D, Sg, Tavg = 277.2, zavg = 1, Tst=288.15, Pst=1.01325E5, EPE=0, Const=137.2364, DEBUG = 0):
    """
    Weymouth(Qdot, Pin, Pout, L, D, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Weymouth formula to generate one of Qdot_st, Pin, Pout - given the other two.
    The unknown is specified by a value of -9999.
    We use PARAMS: [D(iameter), L(ength), Sg].
    Qdot in standard cubic metres per second.
    """
    #Only one of Qdot, Pin, Pout, L should be -9999
    import math
    MainArgs = [Qdot, Pin, Pout, L]
    PARAMS = [D, Sg]
    if not (MainArgs.count(-9999) == 1):
        print("Error - only one argument should be -9999")
        return -9999
    T1 = 137.2364 * (Tst/Pst)*pow(D,8/3)
    denom = Sg * Tavg *zavg
    if Qdot == -9999 :
        if DEBUG == 1 : print("Weymouth to find Qdot. Pin: ", '{:12.3e}'.format(Pin), " Pout: ", '{:12.3e}'.format(Pout))
        T2 = (pow(Pin,2) - pow(Pout,2) - EPE)/(denom*L)
        Qst = T1*pow(T2,1/2)
        if DEBUG == 1 : print("In Weymouth [Qst]: ", Qst)
        return Qst
    elif Pin == -9999:
        if DEBUG == 1 : print("Weymouth to find Pin. Qdot: ", '{:12.3e}'.format(Qdot), " Pout: ", '{:12.3e}'.format(Pout))
        PinSq = L*denom*(Qdot/T1)**2 + Pout**2 + EPE
        Pin = math.sqrt(PinSq)
        if DEBUG == 1 : print("In Weymouth [Pin]: ", Qdot,Pin)
        return Pin
    elif Pout == -9999:
        PoutSq = Pin**2 - L*denom*(Qdot/T1)**2 - EPE
        if DEBUG == 1 : print("Weymouth to find Pout. Qdot: ", '{:12.3e}'.format(Qdot),
                              " Pin: ", '{:12.3e}'.format(Pin)," Pout: ", '{:12.3e}'.format(Pout))
        Pout = math.sqrt(max(0,PoutSq))#Do I need an warning here?
        return Pout
    elif L == -9999:
        T2 = (pow(Pin,2) - pow(Pout,2) - EPE)/denom
        L = (T1/Qdot)**2 *T2
        return L
    return "ERROR"


def GenLeak(Qdotin, Pin, LLeak, D, Sg, Tavg = 277.2, zavg = 1, Tst=288.15, Pst=1.01325E5, EPE=0, Const=137.2364, DEBUG = 0):
    """
    GenLeak(Qdot, Pin, LLeak, D, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Weymouth formula to generate one of Qdot_st, Pin, Pout - given the other two.
    The unknown is specified by a value of -9999.
    We use PARAMS: [D(iameter), L(ength), Sg].
    Qdot in standard cubic metres per second.
    """
    print("Needs to be written and validated!")
    return [Qout,PLeak,Pout]


In [ ]:
#Some testing :)
Min = 34.25483928234368*0.8 # 0.8 kg/m^3 is from data sheet. Assuming incompressibility.
Mout = Min *1 # A 0% leak :-)
Pin = 9.0E6
Pout = 2.0E6
[MLeak,PLeak,LLeak] = FindLeakInit(Min, Pin, Mout, Pout,
                               D = 0.34, L = 1.65E5, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325e5, EPE = 0, DEBUG = 0)
print("FLInit - MLeak, PLeak, LLeak: ",[MLeak,PLeak,LLeak])
print("-------------------------------")
Min = 34.25483928234368*0.8 # 0.8 kg/m^3 is from data sheet. Assuming incompressibility.
Mout = Min *0.99 # A 1% leak :-)
Pin = 9.0E6
Pout = 2.0E6
[MLeak,PLeak,LLeak] = FindLeakInit(Min, Pin, Mout, Pout,
                               D = 0.34, L = 1.65E5, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325e5, EPE = 0, DEBUG = 0)
print("FLInit - MLeak, PLeak, LLeak: ",[MLeak,PLeak,LLeak])
print("-------------------------------")
print("This just validates a possibly superflous initialisation routine!")
Min = 34.25483928234368*0.8 # 0.8 kg/m^3 is from data sheet. Assuming incompressibility.
Mout = Min *0.99 # A 1% leak :-)
Pin = 9.0E6
Pout = 2.0E6*1.01 # Move the leak away from the outlet
[MLeak,PLeak,LLeak] = FindLeakInit(Min, Pin, Mout, Pout,
                               D = 0.34, L = 1.65E5, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325e5, EPE = 0, DEBUG = 0)
print("FLInit - MLeak, PLeak, LLeak: ",[MLeak,PLeak,LLeak])

NameError: name 'FindLeakInit' is not defined

In [ ]:
from scipy import optimize

#DEFINE Generalised Transport Equations as RetCode = f(Min, Mout, Pin, Pout; parameters)
#Syntax: OUTPUT varb is initialised to -9999.
#This works because, in real life everything must be positive.
# Triple:{Qdot, Pin, Pout} legal example: {-9999, Pin>Pout, Pout>0}- find Qdot, {Qin>0,-Pin,-9999} find Pout by varying Pin
def FindLoc(LLeak, PARAMS):
    """
    FindLoc(Lleak) PARAMS = [Min, Pin, Mout, Pout, LLeak,
        L=165000, D = 0.34, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0, DEBUG = 0]
    - use the flow formula (Weymouth) formula to find Lleak given Min, Mout, Pin, Pout [and the other parameters].
    Min and Mout are in kg/sec .
    """

    [Min, Pin, Mout, Pout, L, D , Sg, Tavg ,zavg, Tst, Pst, EPE , DEBUG]=PARAMS
    #if DEBUG == 1: print("In FindLoc - LLeak", LLeak,"PLeak1/2: ",PLeak1,PLeak2)
    rholeak=rhoin=rhoout=1
    Qdotin = Min/rhoin
    Qdotout = Mout/rhoout
    #Strategies:
    # 1 Find PLeak1, PLeak2 return (PLeak1-PLeak2) {Fail - LLeak goes negative}
    # 2 Find PLeak1 then use PLeak1 & LLeak to find PoutTemp return (Pout-PoutTemp)
    #From the inlet to the leak
    #if DEBUG == 1: print("In FindLoc - LLeak", LLeak,"PLeak1/2: ",PLeak1,PLeak2)
    PLeak1 = Weymouth(Qdotin, Pin, -9999, LLeak, D, Sg, Tavg, zavg, Tst, Pst, EPE, DEBUG = 0)
    #From the leak to the outlet
    #if DEBUG == 1: print("In FindLoc - LLeak", LLeak,"PLeak1: ",PLeak1)
    PoutEst = Weymouth(Qdotout, PLeak1, -9999, (L-LLeak), D,Sg,Tavg,zavg,Tst,Pst,EPE,DEBUG = 0)

    error = Pout-PoutEst
    if DEBUG == 1:
        print("FindLoc - LLeak", '{:9.2e}'.format(LLeak[0]),
              "PLeak1:",'{:9.2e}'.format(PLeak1),"PoutEst:",'{:9.2e}'.format(PoutEst),
             " Error:",'{:9.2e}'.format(error))
    return error


def FindLeak(Min, Pin, Mout, Pout, LLeak0, L= 165000, D = 0.34, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0, DEBUG = 0):
    """
    FindLeak(Min, Pin, Mout, Pout, LLeak0,
        L=165000, D = 0.34, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0, DEBUG = 0)
    - use the flow formula (Weymouth) formula to generate MLeak, PLeak and LLeak.
    We use PARAMS: [D(iameter), L(ength), Sg].
    Min and Mout are in kg/sec in standard cubic metres per second.
    """
    print("Start of FindLeak - Mout:",Mout)
    rhoin = 1
    PARAMS = [Min, Pin, Mout, Pout, L, D, Sg, Tavg, zavg, Tst, Pst, EPE, DEBUG]
    MLeak = Min - Mout
    LLeakOpt = optimize.fsolve(FindLoc, LLeak0, PARAMS, full_output=0, xtol=1.49012e-08)
    PLeak = Weymouth(Min/rhoin, Pin, -9999, LLeakOpt, D, Sg, Tavg, zavg, Tst, Pst, EPE, DEBUG = 0)
    print("End of FindLeak - Mout,LLeakOpt:",Mout,LLeakOpt)
    return [MLeak,PLeak,LLeakOpt]
def CheckLeak(Pin,PLeak,Pout,LLeak,L = 165000,D = 0.34,Sg = 0.693, rhoin = 1):
    """
    Usage: CheckLeak(Pin,PLeak,Pout,LLeak,L = 165000,D = 0.34,Sg = 0.693, rhoin = 1)
        computes the forward problem - given the ("two") lengths and three pressures -
        what is the mass flow of the leak : THE ANSWER verifying FindLeak()
    """
    Qin = Weymouth(-9999,Pin, PLeak, LLeak, D, Sg)
    print("L-LLeak: ",L-LLeak,"Ratio(Lleak/L): ",LLeak/L)
    Qout = Weymouth(-9999,PLeak, Pout, (L-LLeak), D, Sg)
    print("In CheckLeak Qin,Qout,Pout",Qin,Qout,Pout)
    MLeakTest = (Qin-Qout)*rhoin
    print("Check - MLeak: ", MLeakTest)#'{:9.3e}'.format(MLeakTest))
    return MLeakTest

In [ ]:
#Generate a case (10%leak middle of the pipe)
PLeakNom = Weymouth(34.25483928234368,9.0E6,-9999,1.65E5/2,0.34,0.693)
PoutNom = Weymouth(34.25483928234368*0.9,PLeakNom,-9999,1.65E5/2,0.34,0.693)
print("10% leak, mid-pipe = PLeakNom, PoutNom: ",PLeakNom,PoutNom)
#

In [ ]:
#FindLeak validation
L = 165000
Min = 34.25483928234368*1 # 0.8 kg/m^3 is from data sheet. Assuming incompressibility.
Mout = Min * 0.9 # A 1% leak :-)
Pin = 9.0E6
Pout = 2.0E6 * 1.3 # Pressure drop is less!
print("Main tests - Mout:",Mout)
[MLeak,PLeak,LLeak] = FindLeak(Min, Pin, Mout, Pout, (0.4*L),
                               D = 0.34, L = 1.65E5, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325e5, EPE = 0, DEBUG = 1)
print("MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
print("Min,Pout:",'{:9.3e}'.format(Min),'{:9.3e}'.format(Pout), "MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
CheckLeak(Pin,PLeak,Pout,LLeak[0])
print("===================")
Min = 34.25483928234368*1 # 0.8 kg/m^3 is from data sheet. Assuming incompressibility.
Mout = Min * 0.8 # A 1% leak :-)
Pout = 2.0E6 * 1.3 # Pressure drop is less!
print("Main - Mout:",Mout)
[MLeak,PLeak,LLeak] = FindLeak(Min, Pin, Mout, Pout, (0.4*L),
                               D = 0.34, L = 1.65E5, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325e5, EPE = 0, DEBUG = 0)
print("MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
print("Min,Pout:",'{:9.3e}'.format(Min),'{:9.3e}'.format(Pout), "MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
CheckLeak(Pin,PLeak,Pout,LLeak[0])
print("===================")
Min = 34.25483928234368*1 # 0.8 kg/m^3 is from data sheet. Assuming incompressibility.
Mout = Min * 0.9 # A 1% leak :-)
Pout = 2.0E6 * 1.4 # Pressure drop is less!
print("Main - Mout:",Mout)
[MLeak,PLeak,LLeak] = FindLeak(Min, Pin, Mout, Pout, (0.4*L),
                               D = 0.34, L = 1.65E5, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325e5, EPE = 0, DEBUG = 0)
print("MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
print("Min,Pout:",'{:9.3e}'.format(Min),'{:9.3e}'.format(Pout), "MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
CheckLeak(Pin,PLeak,Pout,LLeak[0])
print("===================")
print("The mid-point case")
Min = 34.25483928234368*1 # 0.8 kg/m^3 is from data sheet. Assuming incompressibility.
Mout = Min * 0.9 # A 1% leak :-)
Pout = 3363777.638310831 # From previous calculation
print("Main - Mout:",Mout)
[MLeak,PLeak,LLeak] = FindLeak(Min, Pin, Mout, Pout, (0.4*L),
                               D = 0.34, L = 1.65E5, Sg = 0.693, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325e5, EPE = 0, DEBUG = 0)
print("MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
print("Min,Pout:",'{:9.3e}'.format(Min),'{:9.3e}'.format(Pout), "MLeak, PLeak, LLeak: ",'{:9.3e}'.format(MLeak), '{:9.3e}'.format(PLeak),'{:9.3e}'.format(LLeak[0]))
CheckLeak(Pin,PLeak,Pout,LLeak[0])

Main tests - Mout: 30.829355354109314
Start of FindLeak - Mout: 30.829355354109314
FindLoc - LLeak  6.60e+04 PLeak1:  7.09e+06 PoutEst:  3.57e+06  Error: -9.75e+05
FindLoc - LLeak  6.60e+04 PLeak1:  7.09e+06 PoutEst:  3.57e+06  Error: -9.75e+05
FindLoc - LLeak  6.60e+04 PLeak1:  7.09e+06 PoutEst:  3.57e+06  Error: -9.75e+05
FindLoc - LLeak  6.60e+04 PLeak1:  7.09e+06 PoutEst:  3.57e+06  Error: -9.75e+05
FindLoc - LLeak  1.45e+05 PLeak1:  3.68e+06 PoutEst:  2.41e+06  Error:  1.90e+05
FindLoc - LLeak  1.32e+05 PLeak1:  4.42e+06 PoutEst:  2.64e+06  Error: -3.53e+04
FindLoc - LLeak  1.34e+05 PLeak1:  4.31e+06 PoutEst:  2.60e+06  Error: -1.29e+03
FindLoc - LLeak  1.34e+05 PLeak1:  4.30e+06 PoutEst:  2.60e+06  Error:  8.73e+00
FindLoc - LLeak  1.34e+05 PLeak1:  4.30e+06 PoutEst:  2.60e+06  Error: -2.16e-03
FindLoc - LLeak  1.34e+05 PLeak1:  4.30e+06 PoutEst:  2.60e+06  Error: -2.79e-09
End of FindLeak - Mout,LLeakOpt: 30.829355354109314 [133872.18045113]
MLeak, PLeak, LLeak:  3.425e+00 4.304

3.4254839282343745

In [ ]:
#SO the N-leaks problem for a single pipe requires least squares solution of the equations.
#Here's the documentation and calling sequence.
#Or maybe it's time to go to Bayesian optimisation.
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html
#scipy.optimize.least_squares
#(fun, x0, jac='2-point', bounds=- inf, inf, method='trf',
# ftol=1e-08, xtol=1e-08, gtol=1e-08, x_scale=1.0, loss='linear', f_scale=1.0, diff_step=None, tr_solver=None, tr_options={},
# jac_sparsity=None, max_nfev=None, verbose=0, args=(), kwargs={})

#### Stage 1.1: Single pipe - *N*-leaks.
Some notation:
1. Define the position of leak $i$ by $L_i$.
   - $L_0$ is the inlet and $L_{N+1}$ is the outlet. Of course, $L_0 = 0$ and the pipe length, $L_{N+1}$ is known. _Unknowns: $N$_
1. We assume that we have data for the flow performance for $R$ different inlet pressures. The data is the inlet and outlet pressures and flow rates:  
$P_0(r) , P_{N+1}(r), \dot{Q}_0(r) , \dot{Q}_{N+1}(r)$ respectively.
_"Equations": 4R (discount)_
1. With each pipe segment, $L_{i-1}$ to $L_i$, associate a flowrate $\dot{Q}_{i-1}(r)$
 $i = 1\dots N, r=1\dots R$
1. With each node ($L_i$) we have a leak coefficient $C_i$. _Unknowns: $N$_
1. Further, for each node and for each flow case ($r$),  we associate a pressure, $P_i(r)$ and a leakage rate, $\dot{Q}_i^{leak}(r)$.
  - Obviously mass conservation at node $i$ gives:
  $$
  \dot{Q}_i^{leak}(r) = \dot{Q}_i(r) - \dot{Q}_{i-1}(r)
  $$

We have the folling relationships:
1. We assume that the leakage rate is determined by Bernoulli's equation; so
$$
\dot{Q}_i^{leak}(r) = C_i \left[P_i(r) - P_{ext}\right]^2
$$
where $P_{ext}$ is the external pressure, assumed consant for values of $i$ and $r$.
1. The pressure drop along segment $L_{i-1}$ to $L_i$ satisfies:
$$
{\cal F}\left(\overline{\rho}_{i-1}(r)\dot{Q}_{i-1}(r),P_{i-1}(r),P_{i}(r),(L_{i-1}-L_i)\right) = 0
$$


The next question - how do we find *N*?
Approach:
- Loop: NLeak = 1,NLkMax monitor size of the normalized residual.
- Compare:
    - Infinity norm (look at some components - e.g. all $P_i$, $Q_i$ and _ALL_ equations).
    - Two norm as we did with PAMG -?- $\left (\frac{1}{NLeak}\Sigma_{NLeak} res_{P_i}^2 \right)$
    -**How does this quantity vary?**
    

In [ ]:
def GenLeak(Qdotin,Pin,LEAKS,D,Sg,Tavg = 277.2,zavg =1,Tst=288.15,Pst=1.01325E5,EPE=0,Const=137.2364,Pext=1.01325E5,DEBUG = 0):
    """
    GenLeak(Qdot, Pin, LEAKS, D, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Weymouth formula to generate one of Qdot_st, Pin, Pout - given the other two.
    The unknown is specified by a value of -9999.
    We use PARAMS: [D(iameter), L(ength), Sg].
    Qdot in standard cubic metres per second.
    """
#Sanity check
    if len(LEAKS) == 0:
#No leaks case (Compute Pout)
        Pout = 0
    if len(LEAKS) > 0:
        if len(LEAKS[0]) != 2:
            print("PANIC")
    print("Needs to be written and validated!")
    return [Qout,LEAKS,Pout]
#TEST CASES:
# Standard case - 0 leaks
# Case 1 leak (investigate the sensitivity - to size of leak and position, e.g. near to ends)
# Case 2 leaks (investigate accuracy and sensitivity)
# Case 10 leaks

#### Stage 2: Single pipes and uncertainty.
-   In the presence of uncertainty we have uncertainty in the sensor readings: $\dot{M}_{in}$, $P_{in}$ and $\dot{M}_{out}$, $P_{out}$
   -  Assume they are all uniformly distributed with a parameter ${\mathbf{\chi}}$.  
   Values of $\chi$ can be obtained from the equipment manufacturer (see above).
- Denote the uncertain readings by: $\overline{\dot{M}}_{x}, \overline{P}_{x}$ at location $x \in \{in, leak, out\}$.
Then we have:
   -  $\overline{\dot{M}}_{leak} = \overline{\dot{M}}_{in} - \overline{\dot{M}}_{out}$
   -  ${\cal W}(\overline{\dot{M}}_{in},\overline{P}_{in},\overline{P}_{leak},\overline{L}_{leak}) = 0$
   -  ${\cal W}(\overline{\dot{M}}_{out},\overline{P}_{leak},\overline{P}_{out},(L - \overline{L}_{leak})) = 0$
   -  $\overline{\dot{M}}_{leak} = C(\overline{P_{leak}} - P_{external})^2$
- Let us assume that we have $n$ uncertainty readings denoted by:  $\overline{\dot{M}}_{x}(i), \overline{P}_{x}(i)$ at location $x \in \{in, leak, out\}$; $ 1 \leq i \leq n$.  
Then, for each $i$ we can write:
   -  $\overline{\dot{M}}_{leak}(i) = \overline{\dot{M}}_{in}(i) - \overline{\dot{M}}_{out}(i)$
   -  ${\cal W}(\overline{\dot{M}}_{in}(i),\overline{P}_{in}(i),\overline{P}_{leak}(i),\overline{L}_{leak}(i)) = 0$
   -  ${\cal W}(\overline{\dot{M}}_{out}(i),\overline{P}_{leak}(i),\overline{P}_{out},(L - \overline{L}_{leak}(i))) = 0$
   -  $\overline{\dot{M}}_{leak}(i) = C(\overline{P_{leak}}(i) - P_{external})^2$
we are assuming that: $C$ and $P_{external}$ are unaffected by the flow paramters.

In [ ]:
print('Some little tests of the len() function:',len([]),len([1,2]),len([[1,2]]))

Some little tests of the len() function: 0 2 1


### Laminar flow in leaky trees

"DIAGRAM"
<img src="https://github.com/mm002a/JupyterNotebooks-V0.0/blob/main/Y-junc.png?raw=1" />




Let's make some simplifications and see where we get to!  
Assume that:
   1. the network is a directed tree.
   1. the flow is laminar so the drag is $64/Re$
   1. all pipes have the same diameter.
   1. that the inlet flow rate at the the root is given: $Q_{in} = Q_0$
   1. all leaf nodes are at the same pressure: $P_{external} = P_e$
   1. all leaks are into an external environment at pressure: $P_{external}$
   1. Define the tree with a connectivity graph:
      - Define the root of the tree to be node $0$
      - Associate with each node $i$:
        1. $Par_i$ The node number of the incoming pipe (Parent) (this is redundant information!)
        1. $C_1, C_2, \dots C_{n_i}$ The node numbers of the children of node $i$.
        - Note that we can number the tree so that $Par_i < C_1 < C_2 < \dots < C_{n_i} \forall i$
   - **Data associated** with  node $i$
      - $Q_i$ The incoming flow rate of fluid  
      - $P_i$ The pressure
      - $L_i$ The length of the incoming pipe
   - Therefore: each node has exactly one incoming pipe (except the root). Denote this by $pipe_i$
   
Start really simple:
   - Consider the smallest tree (a "Y" structure).